In [ ]:
import pandas as pd

In [ ]:
asset = pd.read_csv("assets_base_data.csv", sep=";")
imbalance_penalty = pd.read_csv("imbalance_penalty.csv", sep=";")
market_index_price = pd.read_csv("market_index_price.csv", sep=";")
measured = pd.read_csv("measured_20241013.csv", sep=";")

In [ ]:
asset.head()

## Trading revenue

In [ ]:
trades = pd.read_json("trades.json")
trades.head()

In [ ]:
def calculate_trading_revenue(row):
    revenue = row["price"] * row["quantity"]
    if row["side"] == "buy": # presumably
        revenue = revenue * -1
    return revenue
trades["revenue"] = trades.apply(calculate_trading_revenue , axis=1)

In [ ]:
trades.groupby("side")["revenue"].sum()

## Invoice revenue

In [ ]:
def calculate_revenue(row):
    if row["price_model"] == "fixed":
        colname = row["metering_point_id"]
        kw = measured[colname].sum()
        revenue = row["price__eur_per_mwh"] * kw / 1000
    else:
        revenue = 0
    return revenue
asset["revenue"]= asset.apply(calculate_revenue, axis=1)
print(asset["revenue"])

## Imbalance

In [ ]:
df_list = []
for i in range(1,5):
    df_list.append(pd.read_json(f"forecasts/a{i}.json"))
forecasts = pd.concat(df_list).reset_index()
